<a href="https://colab.research.google.com/github/zqian/google_colab_scripts/blob/main/Rubric_Item_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on this [Canvas Idea Conversation](https://community.canvaslms.com/t5/Idea-Conversations/Export-aggregate-rubric-scores-to-identify-strengths-and/idi-p/370265/page/3?emcs_t=S2h8ZW1haWx8ZGlnZXN0X25vdGlmaWNhdGlvbnxLWU9HSEw3TFIzVVpFOHwtMXxPVEhFUlN8aEs#comments), here is a PoC script to generate a csv file that shows the rubric line item scores for all student submissions.

**Install Python Libraries**

In [ ]:
# install all libraries
!pip install canvasapi
!pip install future-fstrings
!pip install pandas

**Step 1: get Canvas credentials**

*   Log into Canvas instance
*   Go to "Account"-> "Settings" -> "+ New Access Token". You can name it as "Rubric Ttem Report", and leave the "Expires" field empty for no expiration. Click on "Generate Token"
*   Copy the token into next section

In [ ]:
from canvasapi import Canvas
from canvasapi.assignment import (
    Assignment,
    AssignmentExtension,
    AssignmentGroup,
    AssignmentOverride,
)

from canvasapi.rubric import Rubric, RubricAssociation
from canvasapi.submission import GroupedSubmission, Submission
import pandas as pd

PERSONAL_ACCESS_TOKEN = 'replace_with_assess_token'
CANVAS_URL = 'replace_with_institution_canvas_url'
COURSE_ID = 'replace_with_canvas_course_id'
ASSIGNMENT_ID = 'replace_with_canvas_assignment_id'

**Step 2: Retrieve assignment and its rubric settings**

In [ ]:
# init CanvasAPI library
# get course, assignment with rubric settings
canvas = Canvas(CANVAS_URL, PERSONAL_ACCESS_TOKEN)
course = canvas.get_course(COURSE_ID)

assignment = course.get_assignment(ASSIGNMENT_ID, include=['overrides'])
assignment_name = assignment.name

# get assignment rubric settings object; retrieve the rubric id
assignment_rubric_settings = assignment.rubric_settings
assignment_rubric_id = assignment_rubric_settings['id']
assignment_rubric_title = assignment_rubric_settings['title']
assignment_rubric_points_possible = assignment_rubric_settings['points_possible']

# get rubric settings
rubric_settings = assignment.rubric
for setting in rubric_settings:
  print(f"{setting['id']} {setting['description']}")

**Step 3: iterating through assignment submissions. Output csv report file for each rubric criteria**

In [ ]:
from google.colab import files

submissions = assignment.get_submissions(include=['rubric_assessment'])

for setting in rubric_settings:
  print(f"{setting['id']} {setting['description']}")
  rubric_item_id = setting['id']
  rubric_item_description = setting['description']

  # the output csv file columns
  # you can modify the column to include other attributes, e.g. student name, points_possible, etc
  df = pd.DataFrame(columns = ["user_id", "points", "comments"])
  for submission in submissions:
    try:
      if submission.rubric_assessment:
        df = df.append({"user_id": submission.user_id,
                  "points": submission.rubric_assessment[rubric_item_id]['points'],
                  "comments": submission.rubric_assessment[rubric_item_id]['comments']}, ignore_index=True)
    except:
      print(f"no rubric assessment for submission {submission}")
  
  # the output file is named after "{assignment_name}_{rubric_item_description}_report.csv"
  # you can change the file name format acoordingly
  download_file_name = f"{assignment_name}_{rubric_item_description}_report.csv"
  df.to_csv(download_file_name, index=False)
  files.download(download_file_name) 
    